In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask-ngrok
!pip install transformers

     |████████████████████████████████| 2.5 MB 14.0 MB/s 
     |████████████████████████████████| 895 kB 57.2 MB/s 
     |████████████████████████████████| 3.3 MB 41.6 MB/s 


In [3]:
import torch
# We'll need these libraries to gather and shape the data.
import requests 
import pandas as pd
from itertools import compress
import re

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

In [4]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")
special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>'}
tokenizer.add_special_tokens(special_tokens_dict)
model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:847: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [5]:

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('/content/drive/MyDrive/sis330/Final/model/model_40.pth'))

<All keys matched successfully>

In [6]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):

    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

def sample_seq(model,abba, context, length, device="cpu", temperature=1, top_k=0, top_p=0.0):

    prompt = f'<BOC>{abba}<EOC>{context}'
    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  #  generated = context
    with torch.no_grad():  
        for _ in range(length):
            inputs = {'input_ids': generated}
            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            next_token_logits = outputs[0][0, -1, :] / temperature
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
    estrofa=tokenizer.decode(generated[0])
    estrofa = estrofa.strip(f'<BOC>{abba}<EOC>')
    estrofa = estrofa.replace('<PAD>','')
    estrofa = estrofa.replace('<PAD','')
    return estrofa
#sentence=sample_seq(model,"abba","tengo miedo de ir a ese lugar.",100)

In [7]:
def generate_parrafo(promt,n):
  mylist = ["aabb", "abab", "abba"]
  a= random.choices(mylist)

  lista = ""
  for i in range(0,n):
    v=sample_seq(model,a,promt,100,'cuda')
    lista +=v+"\n"
  parrafos=lista.split("\n")
  return parrafos

In [ ]:
print(sentence.split('\n'))

['tengo miedo de ir a ese lugar.', 'por donde dejando de ti una nota.', 'aún resonaba el eco de tu acento.', 'Señor y el firmamento.', '']


In [ ]:
sente=generate_parrafo("la vida es tan dulce",10)
print(len(sente))

51


In [ ]:
sente

['la vida es tan dulce y tan bonita.',
 'y entonces amor no entra en ti sonando.',
 'amor que es para ti la flor de mi pensamiento.',
 'y a ti me paré horizonte.',
 '',
 'la vida es tan dulce y largo.',
 'Y dize el pastorcito Ay desdichado.',
 'de aquel que de mi amor a hecho ausencia.',
 'y no quiere gozar la mi presencia.',
 '',
 'la vida es tan dulce que no te dandoé la despedida.',
 'Ya no habré de gozar la sombra de mi vida.',
 'que es al morir dividido y al fin de tu vida.',
 'Es el momento en que te dará el gran momento.',
 '',
 'la vida es tan dulce yundo.',
 'que mis pobres Serrano.',
 'ha vivido a la luz de la inocencia.',
 'y al resplandor de la inocencia.',
 '',
 'la vida es tan dulce y suave.',
 'que he sido por tu amor querido.',
 ' Aguándome dulce consejo.',
 'en tal caso de mí no te olvidaras.',
 '',
 'la vida es tan dulce y entera.',
 'que no temo que no temo que no temo.',
 'A las doce de la noche por las puertas.',
 'de la noble palombacho escrita.',
 '',
 'la vida e

In [ ]:
sente=sente.split("\n")
print(sente)

['la vida es tan dulce y olor tan olifante.', 'que muchas veces al infierno igualo.', 'por raro al mundo su valor señalo.', 'por falso al hombre su rigor condeno.', 'la vida es tan dulce y transparente.', 'que perdonando a su pena.', 'tan larga y prósperos.', 'paisaje de la gente.', '']


In [8]:
import os

if not os.path.exists('templates'):
  os.makedirs('templates')
if not os.path.exists('static'):
  os.makedirs('static')

!pip install flask_ngrok

from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

In [12]:
index="""
<html>
  <head>
  <style type="text/css">
          div.parrafos {
              text-align: center;
              font-family: "Latin Modern Roman 10";
              font-style: oblique;
              font-size: xx-large;

          }

          div {
              border-radius: 5px;
              background-color: #f2f2f2;
              padding: 20px;
          }

          input[type=text],
          select {
              width: 100%;
              padding: 12px 20px;
              margin: 8px 0;
              display: inline-block;
              border: 1px solid #ccc;
              border-radius: 4px;
              box-sizing: border-box;
          }

          input[type=submit] {
              width: 100%;
              background-color: #33658A;
              color: white;
              padding: 14px 20px;
              margin: 8px 0;
              border: none;
              border-radius: 4px;
              cursor: pointer;
          }

        
         </style>
  </head>
<body>

<div>
<h3>Modelo gpt2</h3>
  <form action="/result" method="POST">
    <label for="palabra">ingresar parrafo</label>
    <input type="text" id="palabra" name="palabra">
    <br>
    Esto puede demorar<br>
    
    <br>
    <input type="submit" value="generar">
  </form>
</div>
</body>
</html>"""
file = open("templates/index.html","w")
file.write(index)
file.close()

result="""
<!doctype html>
<html>
  <head>
      <style type="text/css">
          div.parrafos {
              text-align: center;
              font-family: "Latin Modern Roman 10";
              font-style: oblique;
              font-size: xx-large;
              
          }
          body{
              background-color: #B4DCE7;
          }
      </style>
  </head>
   <body>
      <div class="parrafos">
            <h1>poemas generado:</h1>
            {% for elem in prediction %}
        
              {{ elem }}<br>

            {% endfor %}
      </div>
   </body>
</html>
"""
file = open("templates/result.html","w")
file.write(result)
file.close()

In [ ]:
import os
import numpy as np
import flask
import pickle
from flask import Flask, render_template, request

#creating instance of the class
app = Flask(__name__)
run_with_ngrok(app)

#to tell flask what url shoud trigger the function index()
@app.route('/')
@app.route('/index')
def index():
    return flask.render_template('index.html')

def ValuePredictor(to_predict_list):

    to_predict_list=lista_versos
    return to_predict_list

@app.route('/result', methods = ['POST'])
def result():
    if request.method == 'POST':
        to_predict_list = request.form.to_dict()
        to_predict_list = list(to_predict_list.values())
        print(to_predict_list[0])
        try:
           
            mylist = ["aabb", "abab", "abba"]
            prediction = generate_parrafo(to_predict_list[0],3)
        
        except ValueError:
            prediction='Error en el formato de los datos'
        
        return render_template("result.html", prediction=prediction)

if __name__=="__main__":

    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e29f396ca05c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Jul/2021 20:31:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 20:31:10] "GET /favicon.ico HTTP/1.1" 404 -


Me peina el viento los cabellos


127.0.0.1 - - [21/Jul/2021 20:33:06] "POST /result HTTP/1.1" 200 -


Me peina el viento los cabellos


127.0.0.1 - - [21/Jul/2021 21:43:39] "GET /result HTTP/1.1" 405 -
127.0.0.1 - - [21/Jul/2021 21:45:03] "POST /result HTTP/1.1" 200 -


Me Desagarro las ideas en mi mente
Me Desagarro las ideas en mi mente


127.0.0.1 - - [21/Jul/2021 21:49:25] "POST /result HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 21:49:28] "POST /result HTTP/1.1" 200 -
